# Azure Machine Learning - Model Package Testing (Local)
Sample notebook demonstrating how to pull an image from a target registry, run locally as web service, and submitting data to the model for scoring via a HTTP request

### Import required packages

In [ ]:
from azureml.core import Workspace, Model, Dataset
import subprocess
import requests
import json
import warnings
warnings.filterwarnings('ignore')

### Get connection to AML workspace and default Key Vault

In [ ]:
ws = Workspace.from_config()
kv = ws.get_default_keyvault()

### Retrieve ACR credentials from key vault
<i>Note:</i> If these credentials have not been previously set you will need to add them using the syntax below:
```
kv.set_secret('acr-username', '<YOUR-ACR-USERNAME>')
kv.set_secret('acr-password', '<YOUR-ACR-PASSWORD>')
kv.set_secret('acr-address', '<YOUR-ACR-ADDRESS>')
```

In [ ]:
acr_username = kv.get_secret('acr-username')
acr_password = kv.get_secret('acr-password')
acr_address = kv.get_secret('acr-address')

### Model package image details
In this sample, model packages are added to a target registry based on model name and model version

In [ ]:
model_name = 'iris-classification'
model_version = '1'
location = f'{acr_address}/{model_name}:{model_version}'
location

### Run model package container locally
Launch the container using subprocess library

In [ ]:
subprocess.check_call(['docker', 'login', acr_address, '-u', acr_username, '-p', acr_password])
subprocess.check_call(['docker', 'pull', location])
subprocess.check_call(['docker', 'run', '-d', '-p', '5001:5001', location])

### Retrieve sample data for testing
Expected format of data sent to end point is...
```
[{'sepal length (cm)':6.4, 'sepal width (cm)':3.1, 'petal length (cm)': 5.5, 'petal width (cm)': 1.8}, 
 {'sepal length (cm)':6.7, 'sepal width (cm)':3.0, 'petal length (cm)': 5.0, 'petal width (cm)': 1.7},
 ..., 
]
```
Data should be stringified before sending to the endpoint for inferencing

In [ ]:
dataset = Dataset.get_by_name(ws, 'Testing_Data')
df = dataset.to_pandas_dataframe()
df.drop(columns=['target'], inplace=True)
records = df.to_json(orient='records')

### Submit data to locally running container for inferencing

In [ ]:
uri = 'http://localhost:5001/score'
headers = {'Content-Type': 'application/json'}
response = requests.post(uri, data=records, headers=headers)
response.json()